# Fence Pointer Performance Analysis

This notebook analyzes the performance of different fence pointer implementations in our LSM tree, focusing especially on the Eytzinger layout implementation.

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configure plot styles
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('deep')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

## Load Benchmark Data

In [ ]:
# Load the benchmark results
results_file = '../benchmarks/fence_bench_results_latest.csv'
scaling_file = '../benchmarks/fence_bench_scaling_latest.csv'
range_sizes_file = '../benchmarks/fence_bench_range_sizes_latest.csv'

# Load the main results
df = pd.read_csv(results_file)

# Display the first few rows
df.head()

## Load Scaling Data

In [ ]:
# Load the scaling results
scaling_df = pd.read_csv(scaling_file)

# Display the scaling data
scaling_df.head()

## Load Range Size Data

In [ ]:
# Load the range size results
range_sizes_df = pd.read_csv(range_sizes_file)

# Display the range size data
range_sizes_df.head()

## Performance Comparison - Point Queries

In [ ]:
# Filter for point query time data
point_query_df = df[(df['QueryType'] == 'Point') & (df['MetricType'] == 'Time')].copy()

# Order implementations for consistent plotting
implementation_order = ['Standard', 'FastLane', 'Eytzinger', 'Basic_Eytzinger', 'SIMD_Eytzinger', 'Full_Eytzinger']
point_query_df['Implementation'] = pd.Categorical(point_query_df['Implementation'], categories=implementation_order, ordered=True)
point_query_df = point_query_df.sort_values('Implementation')

# Create a bar chart for point query performance
plt.figure(figsize=(14, 8))
sns.barplot(x='Implementation', y='Value', data=point_query_df, palette='deep')
plt.title('Point Query Performance of Fence Pointer Implementations', fontsize=16)
plt.xlabel('Implementation', fontsize=14)
plt.ylabel('Time per Query (ns)', fontsize=14)
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add values on top of the bars
for i, p in enumerate(plt.gca().patches):
    plt.gca().annotate(f'{float(p.get_height()):.2f}', 
                        (p.get_x() + p.get_width() / 2., p.get_height()), 
                        ha = 'center', va = 'bottom', 
                        xytext = (0, 5), textcoords = 'offset points')

plt.tight_layout()
plt.savefig('../benchmarks/fence_pointer_point_query.png', dpi=300, bbox_inches='tight')
plt.show()

## Performance Comparison - Range Queries

In [ ]:
# Filter for range query time data
range_query_df = df[(df['QueryType'] == 'Range') & (df['MetricType'] == 'Time')].copy()

# Create a bar chart for range query performance
plt.figure(figsize=(14, 8))
ax = sns.barplot(x='Implementation', y='Value', data=range_query_df, palette='deep')
plt.title('Range Query Performance of Fence Pointer Implementations', fontsize=16)
plt.xlabel('Implementation', fontsize=14)
plt.ylabel('Time per Range Query (ns)', fontsize=14)
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add values on top of the bars
for i, p in enumerate(plt.gca().patches):
    plt.gca().annotate(f'{float(p.get_height()):.2f}', 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha = 'center', va = 'bottom', 
                      xytext = (0, 5), textcoords = 'offset points')

plt.tight_layout()
plt.savefig('../benchmarks/fence_pointer_range_query.png', dpi=300, bbox_inches='tight')
plt.show()

## Improvement Over Standard Implementation

In [ ]:
# Filter for improvement data
improvement_df = df[df['MetricType'] == 'Improvement'].copy()

# Create a consistent order for implementations
implementation_order = ['FastLane', 'Eytzinger', 'Basic_Eytzinger', 'SIMD_Eytzinger', 'Full_Eytzinger']
improvement_df['Implementation'] = pd.Categorical(improvement_df['Implementation'], categories=implementation_order, ordered=True)
improvement_df = improvement_df.sort_values(['QueryType', 'Implementation'])

# Create a grouped bar chart for improvements
plt.figure(figsize=(14, 8))
sns.barplot(x='Implementation', y='Value', hue='QueryType', data=improvement_df, palette='deep')
plt.title('Performance Improvement Over Standard Fence Pointers', fontsize=16)
plt.xlabel('Implementation', fontsize=14)
plt.ylabel('Improvement (%)', fontsize=14)
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12)
plt.axhline(y=0, color='r', linestyle='--')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(title='Query Type', fontsize=12, title_fontsize=12)

# Add values on top of the bars
for i, p in enumerate(plt.gca().patches):
    plt.gca().annotate(f'{float(p.get_height()):.1f}%', 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha = 'center', va = 'bottom' if p.get_height() >= 0 else 'top', 
                      xytext = (0, 5 if p.get_height() >= 0 else -15), textcoords = 'offset points')

plt.tight_layout()
plt.savefig('../benchmarks/fence_pointer_improvement.png', dpi=300, bbox_inches='tight')
plt.show()

## Memory Usage Comparison

In [ ]:
# Filter for memory ratio data
memory_df = df[df['MetricType'] == 'MemoryRatio'].copy()

# Create a bar chart for memory usage
plt.figure(figsize=(12, 8))
ax = sns.barplot(x='Implementation', y='Value', data=memory_df, palette='deep')
plt.title('Memory Usage Relative to Standard Fence Pointers', fontsize=16)
plt.xlabel('Implementation', fontsize=14)
plt.ylabel('Memory Ratio (x Standard)', fontsize=14)
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12)
plt.axhline(y=1.0, color='r', linestyle='--', label='Standard Memory Usage')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add values on top of the bars
for i, p in enumerate(plt.gca().patches):
    plt.gca().annotate(f'{float(p.get_height()):.2f}x', 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha = 'center', va = 'bottom', 
                      xytext = (0, 5), textcoords = 'offset points')

plt.tight_layout()
plt.savefig('../benchmarks/fence_pointer_memory.png', dpi=300, bbox_inches='tight')
plt.show()

## Scaling with Dataset Size

In [ ]:
# Create a line chart to show scaling with dataset size
plt.figure(figsize=(14, 10))

# Convert Size to numeric and set as index
scaling_df['Size'] = pd.to_numeric(scaling_df['Size'])
scaling_df = scaling_df.set_index('Size')

# Plot the three implementation lines
plt.plot(scaling_df.index, scaling_df['Standard_ns'], 'o-', linewidth=2, label='Standard')
plt.plot(scaling_df.index, scaling_df['FastLane_ns'], 's-', linewidth=2, label='FastLane')
plt.plot(scaling_df.index, scaling_df['Eytzinger_ns'], '^-', linewidth=2, label='Eytzinger')

# Set x-axis to log scale for clearer visualization
plt.xscale('log')
plt.yscale('log')

# Add labels and title
plt.title('Fence Pointer Performance Scaling with Dataset Size', fontsize=16)
plt.xlabel('Dataset Size (elements)', fontsize=14)
plt.ylabel('Time per Point Query (ns)', fontsize=14)
plt.xticks(scaling_df.index, [f'{s:,}' for s in scaling_df.index], fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, which="both", ls="-", alpha=0.3)
plt.legend(fontsize=12)

# Add data points
for i, size in enumerate(scaling_df.index):
    plt.annotate(f'{scaling_df.loc[size, "Standard_ns"]:.2f}', 
                 (size, scaling_df.loc[size, "Standard_ns"]),
                 textcoords="offset points", xytext=(5, 5), ha='left')
    plt.annotate(f'{scaling_df.loc[size, "FastLane_ns"]:.2f}', 
                 (size, scaling_df.loc[size, "FastLane_ns"]),
                 textcoords="offset points", xytext=(5, 5), ha='left')
    plt.annotate(f'{scaling_df.loc[size, "Eytzinger_ns"]:.2f}', 
                 (size, scaling_df.loc[size, "Eytzinger_ns"]),
                 textcoords="offset points", xytext=(5, 5), ha='left')

plt.tight_layout()
plt.savefig('../benchmarks/fence_pointer_scaling.png', dpi=300, bbox_inches='tight')
plt.show()

## Performance by Range Size

In [ ]:
# Create a bar chart for performance by range size
plt.figure(figsize=(12, 8))

# Create a twin-axis plot for both time and improvement
ax1 = plt.gca()
ax2 = ax1.twinx()

# Plot time data on the first axis
sns.barplot(x='RangeSize', y='Standard_ns', data=range_sizes_df, color='steelblue', alpha=0.7, ax=ax1, label='Standard')
sns.barplot(x='RangeSize', y='Eytzinger_ns', data=range_sizes_df, color='forestgreen', alpha=0.7, ax=ax1, label='Eytzinger')

# Plot improvement data on the second axis
sns.pointplot(x='RangeSize', y='Improvement_pct', data=range_sizes_df, color='red', ax=ax2, label='Improvement (%)')

# Set labels and title
ax1.set_title('Range Query Performance by Range Size', fontsize=16)
ax1.set_xlabel('Range Size Category', fontsize=14)
ax1.set_ylabel('Time per Range Query (ns)', fontsize=14)
ax2.set_ylabel('Improvement (%)', fontsize=14, color='red')
ax1.tick_params(axis='x', labelsize=12)
ax1.tick_params(axis='y', labelsize=12)
ax2.tick_params(axis='y', labelsize=12, colors='red')

# Add a legend
handles1, labels1 = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(handles1 + handles2, labels1 + labels2, loc='upper left', fontsize=12)

# Add data annotations
for i, row in range_sizes_df.iterrows():
    ax1.annotate(f'{row["Standard_ns"]:.2f}', 
               (i-0.2, row["Standard_ns"]/2), 
               ha='center', color='white', fontweight='bold')
    ax1.annotate(f'{row["Eytzinger_ns"]:.2f}', 
               (i+0.2, row["Eytzinger_ns"]/2), 
               ha='center', color='white', fontweight='bold')
    ax2.annotate(f'{row["Improvement_pct"]:.1f}%', 
               (i, row["Improvement_pct"]+5), 
               ha='center', va='bottom', color='red')

plt.tight_layout()
plt.savefig('../benchmarks/fence_pointer_range_sizes.png', dpi=300, bbox_inches='tight')
plt.show()

## Optimization Impact Analysis

In [ ]:
# Filter for the optimization comparisons
optimization_df = df[(df['Implementation'].isin(['Basic_Eytzinger', 'SIMD_Eytzinger', 'Full_Eytzinger'])) & 
                     (df['MetricType'] == 'Time')].copy()

# Create a more readable version of the implementation names
implementation_mapping = {
    'Basic_Eytzinger': 'Basic Eytzinger',
    'SIMD_Eytzinger': 'SIMD-Only',
    'Full_Eytzinger': 'Fully Optimized'
}
optimization_df['Implementation'] = optimization_df['Implementation'].map(implementation_mapping)

# Order implementations for consistent plotting
implementation_order = ['Basic Eytzinger', 'SIMD-Only', 'Fully Optimized']
optimization_df['Implementation'] = pd.Categorical(optimization_df['Implementation'], categories=implementation_order, ordered=True)
optimization_df = optimization_df.sort_values(['QueryType', 'Implementation'])

# Create a grouped bar chart for optimization impact
plt.figure(figsize=(14, 8))
sns.barplot(x='Implementation', y='Value', hue='QueryType', data=optimization_df, palette='deep')
plt.title('Impact of Optimizations on Eytzinger Fence Pointers', fontsize=16)
plt.xlabel('Implementation', fontsize=14)
plt.ylabel('Time per Query (ns)', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(title='Query Type', fontsize=12, title_fontsize=12)

# Add values on top of the bars
for i, p in enumerate(plt.gca().patches):
    plt.gca().annotate(f'{float(p.get_height()):.2f}', 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha = 'center', va = 'bottom', 
                      xytext = (0, 5), textcoords = 'offset points')

plt.tight_layout()
plt.savefig('../benchmarks/fence_pointer_optimization_impact.png', dpi=300, bbox_inches='tight')
plt.show()

## Summary Table for Report

In [ ]:
# Create a summary table for point queries
point_summary = df[(df['QueryType'] == 'Point') & (df['MetricType'].isin(['Time', 'Improvement']))].copy()
point_summary = point_summary.pivot(index='Implementation', columns='MetricType', values='Value')
point_summary = point_summary.reset_index()

# Create a summary table for range queries
range_summary = df[(df['QueryType'] == 'Range') & (df['MetricType'].isin(['Time', 'Improvement']))].copy()
range_summary = range_summary.pivot(index='Implementation', columns='MetricType', values='Value')
range_summary = range_summary.reset_index()

# Create a memory usage summary
memory_summary = df[df['MetricType'] == 'MemoryRatio'].copy()
memory_summary = memory_summary[['Implementation', 'Value']].rename(columns={'Value': 'MemoryRatio'})

# Merge all summaries
summary = pd.merge(point_summary, range_summary, on='Implementation', suffixes=('_Point', '_Range'))
summary = pd.merge(summary, memory_summary, on='Implementation', how='left')

# Format the summary for the report
summary_formatted = summary.copy()
summary_formatted['Time_Point'] = summary_formatted['Time_Point'].apply(lambda x: f'{float(x):.2f} ns' if not pd.isna(x) else 'N/A')
summary_formatted['Time_Range'] = summary_formatted['Time_Range'].apply(lambda x: f'{float(x):.2f} ns' if not pd.isna(x) else 'N/A')
summary_formatted['Improvement_Point'] = summary_formatted['Improvement_Point'].apply(lambda x: f'{float(x):+.1f}%' if not pd.isna(x) else 'N/A')
summary_formatted['Improvement_Range'] = summary_formatted['Improvement_Range'].apply(lambda x: f'{float(x):+.1f}%' if not pd.isna(x) else 'N/A')
summary_formatted['MemoryRatio'] = summary_formatted['MemoryRatio'].apply(lambda x: f'{float(x):.2f}x' if not pd.isna(x) else '1.00x')

# Rename columns for clarity
summary_formatted = summary_formatted.rename(columns={
    'Implementation': 'Implementation',
    'Time_Point': 'Point Query Time',
    'Time_Range': 'Range Query Time',
    'Improvement_Point': 'Point Improvement',
    'Improvement_Range': 'Range Improvement',
    'MemoryRatio': 'Memory Usage Ratio'
})

# Display and save the summary
display(summary_formatted)
summary_formatted.to_csv('../benchmarks/fence_pointer_summary.csv', index=False)

## Key Statistics for the Report

In [ ]:
# Extract key statistics for the report
# Best point query performance
point_best_time = point_query_df.loc[point_query_df['Value'].idxmin()]
point_best_impl = point_best_time['Implementation']
point_best_time_val = point_best_time['Value']

# Best range query performance
range_best_time = range_query_df.loc[range_query_df['Value'].idxmin()]
range_best_impl = range_best_time['Implementation']
range_best_time_val = range_best_time['Value']

# Eytzinger vs Standard improvement
eytzinger_point_improvement = df[(df['Implementation'] == 'Eytzinger') & 
                                (df['QueryType'] == 'Point') & 
                                (df['MetricType'] == 'Improvement')]['Value'].values[0]
eytzinger_range_improvement = df[(df['Implementation'] == 'Eytzinger') & 
                                (df['QueryType'] == 'Range') & 
                                (df['MetricType'] == 'Improvement')]['Value'].values[0]

# Memory efficiency
eytzinger_memory = df[(df['Implementation'] == 'Eytzinger') & 
                     (df['MetricType'] == 'MemoryRatio')]['Value'].values[0]

# Best improvement for point and range queries
best_point_improvement = improvement_df.loc[improvement_df[improvement_df['QueryType'] == 'Point']['Value'].idxmax()]
best_range_improvement = improvement_df.loc[improvement_df[improvement_df['QueryType'] == 'Range']['Value'].idxmax()]

# Print key statistics
print(f"Best point query implementation: {point_best_impl} ({point_best_time_val:.2f} ns)")
print(f"Best range query implementation: {range_best_impl} ({range_best_time_val:.2f} ns)")
print(f"Eytzinger vs Standard point queries: {eytzinger_point_improvement:+.1f}%")
print(f"Eytzinger vs Standard range queries: {eytzinger_range_improvement:+.1f}%")
print(f"Eytzinger memory ratio: {eytzinger_memory:.2f}x Standard")
print(f"Best point query improvement: {best_point_improvement['Implementation']} ({best_point_improvement['Value']:+.1f}%)")
print(f"Best range query improvement: {best_range_improvement['Implementation']} ({best_range_improvement['Value']:+.1f}%)")

# Generate key stats CSV for the report
key_stats = {
    "Metric": [
        "Best Point Query Implementation",
        "Best Range Query Implementation",
        "Eytzinger Point Query Improvement",
        "Eytzinger Range Query Improvement",
        "Eytzinger Memory Usage",
        "Best Point Query Implementation",
        "Best Range Query Implementation"
    ],
    "Value": [
        f"{point_best_impl} ({point_best_time_val:.2f} ns)",
        f"{range_best_impl} ({range_best_time_val:.2f} ns)",
        f"{eytzinger_point_improvement:+.1f}%",
        f"{eytzinger_range_improvement:+.1f}%",
        f"{eytzinger_memory:.2f}x Standard",
        f"{best_point_improvement['Implementation']} ({best_point_improvement['Value']:+.1f}%)",
        f"{best_range_improvement['Implementation']} ({best_range_improvement['Value']:+.1f}%)"
    ]
}

key_stats_df = pd.DataFrame(key_stats)
key_stats_df.to_csv('../benchmarks/fence_pointer_key_stats.csv', index=False)
display(key_stats_df)